__This notebook includes multiple tests and approches to try and use SHAP on iPPG2BP network, to understand it and to visualize data.__

In [1]:
import scipy.io
import scipy.signal
import numpy as np
import tensorflow as tf
import keras
import cv2
import shap
import matplotlib.pyplot as plt

from lime import lime_image
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor
from keras.models import Model, Sequential
from keras.layers import Conv2D, Input, Dense, MaxPool2D, Flatten
from sklearn.metrics import classification_report
from tensorflow.python.ops.numpy_ops import np_config
from skimage.segmentation import mark_boundaries

C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://

Prediction with pre-trained model  
---------------------------------

In [2]:
# We need to disable eager execution for Adam
# optimizer to work.
tf.compat.v1.disable_eager_execution()

In [3]:
data = scipy.io.loadmat("data_test.mat")
bp_test = data['CWT_bp_test']
ppg_test = data['CWT_ppg_test']

In [5]:
xtest = np.zeros((ppg_test.shape[1], ppg_test[0,0]['cfs'][0,0].shape[0], ppg_test[0,0]['cfs'][0,0].shape[1],2))
ytest = np.zeros((bp_test.shape[1], bp_test[0,0]['cfs'][0,0].shape[0], bp_test[0,0]['cfs'][0,0].shape[1],2))

for i in range(data['CWT_ppg_test'].shape[1]):
    xtest[i,:,:,0] = np.real(ppg_test[0,i]['cfs'][0,0])
    xtest[i,:,:,1] = np.imag(ppg_test[0,i]['cfs'][0,0])
    ytest[i,:,:,0] = np.real(bp_test[0,i]['cfs'][0,0])
    ytest[i,:,:,1] = np.imag(bp_test[0,i]['cfs'][0,0])

In [6]:
json_model = open("model.json", "r")
read_json_model = json_model.read()
model = keras.models.model_from_json(read_json_model)

classification_models.models._common_blocks is not loaded, but a Lambda layer uses it. It may cause errors.


Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.load_weights("weights.h5")
model.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-3),
              loss = "mean_squared_error")

In [8]:
ypred = model.predict(xtest)
scipy.io.savemat("results.mat", {"results":ypred})

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


Analysis of BP signals' wavelets   
--------------------------------

Loading bp signals from matlab.   
bp_test signals are computed by applying ''icwt'' matlab function on bp wavelets test data.   
bp_pred signals are computed by applying the same function on predicted wavelets ''ypred'' saved as ''results''.

In [9]:
bp_signals = scipy.io.loadmat("bp_signals.mat")
bp_test = bp_signals["bp_test"][0]
bp_test = np.array([bp_test[i]["bp"][0][0][0] for i in range(bp_test.size)])

bp_pred = bp_signals["bp_results"][0]
bp_pred = np.array([bp_pred[i]["bp"][0][0][0] for i in range(bp_pred.size)])

Displaying test and predicted BP signals.

In [10]:
plt.style.use("bmh")
fig, axs = plt.subplots(len(bp_test)//2, 2, figsize = (8, 10))
axs.shape
for i, (s_test, s_pred) in enumerate(zip(bp_test, bp_pred)):
    axs[i%5, i%2].plot(s_test, label = "test")
    axs[i%5, i%2].plot(s_pred, label = "prediction")
    axs[i%5, i%2].legend(fontsize = "xx-small", loc = "lower right")
fig.tight_layout()
plt.show()

The diffence between test and predicted signals might suggest an error when using inverse wavelet transform, or a bad prediction (very unlikely considering results shown by the network during the study)

In [11]:
data_ppg = scipy.io.loadmat("data_test.mat")['CWT_ppg_test']
ppg = np.zeros((data_ppg[0, 0]['cfs'][0, 0].shape[0], data_ppg[0, 0]['cfs'][0, 0].shape[1], 2))
ppg[:, :, 0] = np.real(data_ppg[0, 0]['cfs'][0, 0])
ppg[:, :, 1] = np.imag(data_ppg[0, 0]['cfs'][0, 0])
ppg = cv2.flip(ppg, 0)

In [13]:
bp_pred = ypred[0]

ppg_real = ppg[:, :, 0]
ppg_imag = ppg[:, :, 1]
mat_ppg = ppg_real+ppg_imag

fig, axs = plt.subplots(2, 3, figsize = (20, 20))

axs[0, 0].matshow(mat_ppg)
axs[0, 0].set_title("Real + Imaginary (ippg)")
axs[0, 0].axis("off")

axs[0, 1].matshow(ppg_real)
axs[0, 1].set_title("Real (ippg)")
axs[0, 1].axis("off")

axs[0, 2].matshow(ppg_imag)
axs[0, 2].set_title("Imaginary (ippg)")
axs[0, 2].axis("off")

axs[1, 0].matshow(bp_pred[:, :, 0]+bp_pred[:, :, 1])
axs[1, 0].set_title("Real + Imaginary (bp pred)")
axs[1, 0].axis("off")

axs[1, 1].matshow(bp_pred[:, :, 0])
axs[1, 1].set_title("Real (bp pred)")
axs[1, 1].axis("off")

axs[1, 2].matshow(bp_pred[:, :, 1])
axs[1, 2].set_title("Imaginary (bp pred)")
axs[1, 2].axis("off")

fig.tight_layout()
plt.show()

I don't see how to use LIME for pixel-to-pixel regression.   
We might want to pick highest weights in the last layer before upsampling and display them, but the image after all convolutions is hardly understandable by the user.

SHAP analysis
-------------

In [17]:
# data_test = np.zeros((data_ppg.shape[1], data_ppg[0, 0]['cfs'][0, 0].shape[0], data_ppg[0, 0]['cfs'][0, 0].shape[1], 2))
# for i in range(data_ppg.shape[1]):
#     data_test[i, :, :, 0] = np.real(data_ppg[0, i]["cfs"][0, 0])
#     data_test[i, :, :, 1] = np.imag(data_ppg[0, i]["cfs"][0, 0])

In [18]:
# explainer = shap.GradientExplainer(model, data_test)
# explainer = shap.DeepExplainer(model, data_test)

Cannot use SHAP because output type is not compatible.

New network: inp = ippg signal wavelets; out = mean of bp sig   
-------------------------------------------------------------

In [72]:
# from keras.layers import Reshape

In [71]:
# We use outputs from iPPG2BP network and feed them through 2 Dense layers
# to predict a single value of mean BP.
# The network needs to be trained with full training dataset so we can't use it

# model.trainable = False

# inp = model.input

# x = model.output
# x = Reshape((256, 256, 2))(x)
# x = Flatten()(x)
# x = Dense(1024)(x)

# out = Dense(1)(x)

# mod = Model(inputs = inp, outputs = out)
# mod.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-3),
#             loss = "mean_squared_error", metrics = ["accuracy"])